# Merged meteorological AGB zonal stats colation

This notebook looks for zonal stats subdirectories and the zonal stats csv outputs within them and concatenates all files into one data frame which is exported as a csv to the output directory.

The following conditions apply:

 - run after Seasonal Biomass Zonal Pipeline.
 - env = base

In [ ]:
import pandas as pd
from glob import glob
import os
from calendar import monthrange
from datetime import datetime

In [ ]:
basal = r"E:\cdu\data\output\slats_tern_c_bio_site_totals_v6.csv"
basal_df = pd.read_csv(basal)

In [ ]:
dir_ = r"E:\cdu\data\zonal_stats\output\gr_meteorological_20221020_1025"
output_dir = r"E:\cdu\data\output\zonal_stats"

In [ ]:
def start_seasonal_date(date_):
    """ extract the end dates of the seasonal image zonal stats."""
    
    year = date_[:4]
    month = date_[4:]
    
    start_date = str(year) + str(month) + "01"
    
    return start_date
    

def end_seasonal_date(date_):
    
    """ extract the start dates of the seasonal image zonal stats."""
    #print("date: ", date_)
    year = str(date_[:4])
    month = str(date_[4:])
    #print("month: ", month)
    
    month_, day_range = monthrange(int(year), int(month))
    end_date = str(year) + str(month) + str(day_range)
    #print(end_date)
    return end_date


def im_date_season(df):
    """Collate start date of image into im_date column"""
    
    st_date_list = []
    e_date_list = []
    for i in df.im_name:
        #print(i)
        list_name = i.split("_")
        date = list_name[-2]
        st_date = date[1:7]
        start_date = start_seasonal_date(st_date)
        st_date_list.append(start_date)
        
        e_date = date[7:] 
        end_date = end_seasonal_date(e_date)
        e_date_list.append(end_date)
        
    df["im_s_date"] = st_date_list
    df["im_e_date"] = e_date_list
    
    return df
        
    
def im_date_annual(df):
    """Collate start date of image into im_date column"""
    
    st_date_list = []
    e_date_list = []
    for i in df.im_name:
        #print(i)
        list_name = i.split("_")
        date = list_name[-2]
        st_date = str(date) + "01"
        start_date = start_seasonal_date(st_date)
        st_date_list.append(start_date)
        
        e_date = str(date) + "12"
        end_date = end_seasonal_date(e_date)
        e_date_list.append(end_date)
        
    df["im_s_date"] = st_date_list
    df["im_e_date"] = e_date_list
    
    return df

    
def convert_to_datetime(df, col_nm_s, col_nm_d):
    
    date_list = []
    for i in df[col_nm_s]:
        #print(i)
        datetime_object = datetime.strptime(str(i), '%Y%m%d')
        date_list.append(datetime_object)
        print(datetime_object)
        #df[col_nm_d] =  pd.to_datetime(df[col_nm_s], format='%Y%m%d.%f')
        #date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    df[col_nm_d] = date_list
    return df        

In [ ]:
basal_df = convert_to_datetime(basal_df, "date", "basal_dt")
basal_df.sort_values(by='basal_dt', inplace=True)

In [ ]:
basal_df.shape

In [ ]:
# extract a list of subdirectories from the qld meterological zonal stats
sub_list = next(os.walk(dir_))[1]
sub_list

zonal_list = []
sub_dir_list = []
for sub_dir in sub_list:
    file_list = []
    #print(sub_dir)
    sub_dir_path = os.path.join(dir_, sub_dir)
    sub_dir_df_list = []
    for file_ in glob(os.path.join(sub_dir_path, "*.csv")):
        if "zonal_stats" in file_:
            sub_dir_list.append(file_)
            print(file_)

            df = pd.read_csv(file_)
            df = convert_to_datetime(df, "im_date", "image_dt")
            df.sort_values(by='image_dt', inplace=True)
            print(df.columns)
            sub_dir_df_list.append(df)
    df1 = pd.concat(sub_dir_df_list)
    
    print(df1.columns)
    # locate min column name and delete values that are missing
    df_columns = df1.columns.tolist()
    
    min_column = [columns for columns in df_columns if "min" in columns]
    print("Min column: ", min_column)
    
        
    df1.dropna(subset = [min_columns], inplace=True)
    
    #merge data with basal datset based on the nearest date to the field data colection
    df1 = pd.merge_asof(basal_df, df1, left_on="basal_dt", right_on= "image_dt", by="site", direction="nearest")
        
    output_path = os.path.join(output_dir, "merged_slats_field_agb_meterological_{0}.csv".format(sub_dir))
    df1.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)
    
    
    zonal_list.append(df1)

In [ ]:
# file_list

In [ ]:
                      
# if len(file_list) > 1:
#     df1 = pd.concat(file_list)
#     # convert to datetime
#     df1 = convert_to_datetime(df1, "im_date", "image_dt")

                
#     #sort df on datetime data
#     df1.sort_values(by='image_dt', inplace=True)
#     print(df1.columns)
#     # locate min column name and delete values that are missing
#     df_columns = dft.columns.tolist()
    
#     min_column = [columns for columns in df_columns if "min" in columns]
#     print("Min column: ", min_column)
    
        
#     dp1_start.dropna(subset = [min_columns], inplace=True)

# else:
#     pass

In [ ]:
# df2 = pd.merge_asof(basal_df, df1, left_on="basal_dt", right_on= "image_dt", by="site", direction="nearest")
# output_path = os.path.join(output_dir, "merged_basal_zonal", "merged_slats_field_agb_metero_{0}.csv".format(sub_dir))
# df2.to_csv(os.path.join(output_path), index=False)
# print("File output to: ", output_path)